<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/langchain_firecrawl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install langchain langchain_openai firecrawl-py faiss-cpu -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00


In [4]:
from langchain_community.document_loaders import FireCrawlLoader

In [20]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from google.colab import userdata

def build_vector_retriever(firecrawl_api_key, scrape_url):
  loader = FireCrawlLoader(api_key=firecrawl_api_key, url=scrape_url, mode="scrape")
  documents = loader.load()
  text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
  docs = text_splitter.split_documents(documents)
  embeddings = OpenAIEmbeddings(api_key=userdata.get("OPENAI_API_KEY"))
  db = FAISS.from_documents(docs, embeddings)

  return db.as_retriever()

In [21]:
retriever = build_vector_retriever(userdata.get("FIRECRAWL_API_KEY"), "https://stablediffusionapi.com/docs/stable-diffusion-api/img2img")

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

def build_qa_chain(retriever):
  prompt = ChatPromptTemplate.from_template("Using the context below to answer user's question. If you can't find information within the context, simply answer I don't know.\n\n <context>{context}</context> <question>{question}</question>")
  model = ChatOpenAI(api_key=userdata.get('OPENAI_API_KEY'))

  chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

  return chain

In [23]:
chain = build_qa_chain(retriever=retriever)

chain.invoke("How to use img2img API in Python")

'To use the img2img API in Python, you can make a POST request to the specified endpoint with the required parameters like prompt, negative prompt, initial image, width, height, and other attributes as shown in the provided code snippet. Make sure to import the necessary libraries like requests and json, then create a payload with the required parameters, and send a POST request to the API endpoint.'